In [1]:
import pandas as pd
import numpy as np
import LR


data_path = "Small-diameter-flow.csv"
df = pd.read_csv(data_path, index_col=0)

y = df["Flow"]
X = np.column_stack([np.ones(y.shape[0]), df['Kinematic'], df['Geometric'], df['Inertial']])




In [2]:

m = LR.LinearRegression()
b = m.fit(X, y)
print(b)

[-2.55979314  0.86871519  3.61041818 -0.75368772]


In [3]:
d = m.dimension(b)
print(d)

3


In [4]:
n = m.sample_size(y)
print(n)

198


In [5]:
SSE = m.SSE(y, X, b)
print(SSE)


1.2238849845911814


In [6]:
var = m.variance(SSE, n, d)
print(var)


0.00630868548758341


In [7]:
S = m.deviation(var)
print(S)

0.07942723391622933


In [8]:
Syy = m.Syy(n, y)
print(Syy)


425.14419302220614


In [9]:
SSR = m.SSR(Syy, SSE)
print(SSR)


423.920308037615


In [10]:
F = m.Fstatistic(SSR, d, var)
print(F)


22398.7658956827


In [11]:
Rsq = m.Rsq(SSR, Syy)
print(Rsq)


0.9971212473210772
